In [3]:
from dataclasses import dataclass, field
from typing import List
from sae_lens.training.config import LanguageModelSAERunnerConfig

@dataclass
class SAETrainConfig(LanguageModelSAERunnerConfig):
    dataset_path: str = 'imagenet_data'
    num_workers: int = 0
    num_epochs: int = 1

    expansion_factor: int = 32
    context_size: int = 257
    d_in: int = 1024
    model_name: str = "laion/CLIP-ViT-L-14-DataComp.XL-s13B-b90K"
    model_base: str = "ViT-L-14"
    model_pretrained: str = "datacomp_xl_s13b_b90k"
    hook_point: str = "blocks.{layer}.hook_mlp_out"
    hook_point_layer: List[int] = field(default_factory=lambda: [23])
    dead_feature_window: int = 5000
    use_ghost_grads: bool = True
    feature_sampling_window: int = 1000
    from_pretrained_path: str = None

    b_dec_init_method: str = "geometric_median"

    lr: float = 0.0004
    l1_coefficient: float = 0.00008
    lr_scheduler_name: str = "constant"
    train_batch_size_tokens: int = 4
    lr_warm_up_steps: int = 5000

    n_batches_in_buffer: int = 4
    store_batch_size: int = 4

    log_to_wandb: bool = True
    wandb_project: str = "openclip_sae_training"
    wandb_entity: str = "willfulbytes"
    wandb_log_frequency: int = 100
    eval_every_n_wandb_logs: int = 10
    run_name: str = None

    device: str = "mps"
    seed: int = 42
    n_checkpoints: int = 10
    checkpoint_path: str = None
    dtype: str = "torch.float32"

In [4]:
import torch
torch.set_grad_enabled(False)

In [5]:
from torch.utils.data import Dataset
from vit_prisma.models.base_vit import HookedViT
from open_clip import tokenize
import datasets
from typing import Any, Iterator, cast
from torch.utils.data import DataLoader

class HFDataset(Dataset):
    def __init__(self, data_location, transforms, image_col, text_col):
        self.dataset = datasets.load_dataset(data_location, split="train")
        self.image_col = image_col
        self.text_col = text_col
        self.transforms = transforms

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Remove the extra dimension by squeezing the tensor
        images = self.transforms(self.dataset[idx][self.image_col], return_tensors="pt")["pixel_values"].squeeze(0)
        texts = tokenize([self.dataset[idx][self.text_col]])[0]
        return images, texts

# Update the collate functions accordingly
def collate_fn(data):
    imgs, _ = zip(*data)
    return torch.stack(imgs, dim=0)

def collate_fn_eval(data):
    imgs, texts = zip(*data)
    return torch.stack(imgs, dim=0), torch.stack(texts, dim=0)


class OpenCLIPActivationsStore:
    """
    Class for streaming tokens and generating and storing activations
    while training SAEs.
    """

    def __init__(
        self,
        config: SAETrainConfig,
        model: HookedViT,
        dataset: torch.utils.data.Dataset,
        eval_dataset: torch.utils.data.Dataset = None,
        num_workers: int = 0,
    ):
        self.config = config
        assert (
            not self.config.normalize_activations
        ), "Normalize activations is currently not implemented for vision, sorry!"
        self.normalize_activations = self.config.normalize_activations
        self.model = model
        self.dataset = dataset
        self.eval_dataset = eval_dataset

        self.image_dataloader = torch.utils.data.DataLoader(
            self.dataset,
            shuffle=True,
            num_workers=num_workers,
            batch_size=self.config.store_batch_size,
            collate_fn=collate_fn,
            drop_last=True,
        )
        self.image_dataloader_eval = torch.utils.data.DataLoader(
            self.eval_dataset,
            shuffle=True,
            num_workers=num_workers,
            batch_size=self.config.store_batch_size,
            collate_fn=collate_fn_eval,
            drop_last=True,
        )

        self.image_dataloader_iter = self.get_batch_tokens_internal()
        self.image_dataloader_eval_iter = self.get_val_batch_tokens_internal()

        self.storage_buffer = self.get_buffer(self.config.n_batches_in_buffer // 2)
        self.dataloader = self.get_data_loader()


    def get_batch_tokens_internal(self):
        """
        Streams a batch of tokens from a dataset.
        """
        device = self.config.device
        while True:
            for data in self.image_dataloader:
                data.requires_grad_(False)
                yield data.to(device)  # 5

    def get_batch_tokens(self):
        return next(self.image_dataloader_iter)

    # returns the ground truth class as well.
    def get_val_batch_tokens_internal(self):
        """
        Streams a batch of tokens from a dataset.
        """
        device = self.config.device
        while True:
            for image_data, labels in self.image_dataloader_eval:
                image_data.requires_grad_(False)
                labels.requires_grad_(False)
                yield image_data.to(device), labels.to(device)

    def get_val_batch_tokens(self):
        return next(self.image_dataloader_eval_iter)

    def get_activations(self, batch_tokens: torch.Tensor, get_loss: bool = False):
        """
        Returns activations of shape (batches, context, num_layers, d_in)
        """
        layers = (
            self.config.hook_point_layer
            if isinstance(self.config.hook_point_layer, list)
            else [self.config.hook_point_layer]
        )
        act_names = [self.config.hook_point.format(layer=layer) for layer in layers]
        hook_point_max_layer = max(layers)

        if self.config.hook_point_head_index is not None:
            layerwise_activations = self.model.run_with_cache(
                batch_tokens,
                names_filter=act_names,
                stop_at_layer=hook_point_max_layer + 1,
            )[1]
            activations_list = [
                layerwise_activations[act_name][:, :, self.config.hook_point_head_index]
                for act_name in act_names
            ]
        else:
            layerwise_activations = self.model.run_with_cache(  ####
                batch_tokens,
                names_filter=act_names,
                stop_at_layer=hook_point_max_layer + 1,
            )[1]
            activations_list = [
                layerwise_activations[act_name] for act_name in act_names
            ]

        # Stack along a new dimension to keep separate layers distinct
        stacked_activations = torch.stack(activations_list, dim=2)

        return stacked_activations

    def get_buffer(self, n_batches_in_buffer: int):
        context_size = self.config.context_size
        batch_size = self.config.store_batch_size
        d_in = self.config.d_in
        total_size = batch_size * n_batches_in_buffer
        num_layers = (
            len(self.config.hook_point_layer)
            if isinstance(self.config.hook_point_layer, list)
            else 1
        )  # Number of hook points or layers

        refill_iterator = range(0, batch_size * n_batches_in_buffer, batch_size)
        # Initialize empty tensor buffer of the maximum required size with an additional dimension for layers
        new_buffer = torch.zeros(
            (total_size, context_size, num_layers, d_in),
            dtype=self.config.dtype,
            device=self.config.device,
        )

        for refill_batch_idx_start in refill_iterator:
            refill_batch_tokens = self.get_batch_tokens()  ######
            refill_activations = self.get_activations(refill_batch_tokens)

            new_buffer[
                refill_batch_idx_start : refill_batch_idx_start + batch_size, ...
            ] = refill_activations

            # pbar.update(1)

        new_buffer = new_buffer.reshape(-1, num_layers, d_in)
        new_buffer = new_buffer[torch.randperm(new_buffer.shape[0])]

        return new_buffer

    def get_data_loader(
        self,
    ) -> Iterator[Any]:
        """
        Return a torch.utils.dataloader which you can get batches from.

        Should automatically refill the buffer when it gets to n % full.
        (better mixing if you refill and shuffle regularly).

        """

        batch_size = self.config.train_batch_size_tokens

        # 1. # create new buffer by mixing stored and new buffer
        mixing_buffer = torch.cat(
            [self.get_buffer(self.config.n_batches_in_buffer // 2), self.storage_buffer], ####
            dim=0,
        )

        mixing_buffer = mixing_buffer[torch.randperm(mixing_buffer.shape[0])]

        # 2.  put 50 % in storage
        self.storage_buffer = mixing_buffer[: mixing_buffer.shape[0] // 2]

        # 3. put other 50 % in a dataloader
        dataloader = iter(
            DataLoader(
                # TODO: seems like a typing bug?
                cast(Any, mixing_buffer[mixing_buffer.shape[0] // 2 :]),
                batch_size=batch_size,
                shuffle=True,
            )
        )

        return dataloader

    def next_batch(self):
        """
        Get the next batch from the current DataLoader.
        If the DataLoader is exhausted, refill the buffer and create a new DataLoader.
        """
        try:
            # Try to get the next batch
            return next(self.dataloader)
        except StopIteration:
            # If the DataLoader is exhausted, create a new one
            self.dataloader = self.get_data_loader() #### 97
            return next(self.dataloader)


In [6]:
from sae_lens.training.sae_group import SparseAutoencoderDictionary
from transformers import CLIPProcessor

config = SAETrainConfig()
processor = CLIPProcessor.from_pretrained(config.model_name)
dataset = HFDataset("awilliamson/fashion-train", processor.image_processor, "image", "text") # load_dataset("awilliamson/fashion-train", split="train")
eval_dataset = HFDataset("awilliamson/fashion-eval", processor.image_processor, "image", "text") # load_dataset("awilliamson/fashion-validation", split="train")
# cfg.training_tokens = int(1_300_000*setup_args['num_epochs']) * cfg.context_size
config.training_tokens = len(dataset) * config.context_size * config.num_epochs
sae_group = SparseAutoencoderDictionary(config)
model = HookedViT.from_pretrained(config.model_name, is_timm=False, is_clip=True)
model.to(config.device)

activation_store = OpenCLIPActivationsStore(
    config = config,
    model = model,
    dataset = dataset,
    eval_dataset = eval_dataset,
    num_workers = 0,
)

for i, (name, sae) in enumerate(sae_group):
    hyp = sae.cfg
    print(
        f"{i}: Name: {name} Layer {hyp.hook_point_layer}, p_norm {hyp.lp_norm}, alpha {hyp.l1_coefficient}"
    )




Run name: 32768-L1-8e-05-LR-0.0004-Tokens-2.000e+06
n_tokens_per_buffer (millions): 0.032896
Lower bound: n_contexts_per_buffer (millions): 0.000128
Total training steps: 500000
Total wandb updates: 5000
n_tokens_per_feature_sampling_window (millions): 1.028
n_tokens_per_dead_feature_window (millions): 5.14
We will reset the sparsity calculation 500 times.
Number tokens in sparsity calculation window: 4.00e+03
Using Ghost Grads.


/Users/aw/projects/vitsearch/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Run name: 32768-L1-8e-05-LR-0.0004-Tokens-3.447e+06
n_tokens_per_buffer (millions): 0.032896
Lower bound: n_contexts_per_buffer (millions): 0.000128
Total training steps: 861785
Total wandb updates: 8617
n_tokens_per_feature_sampling_window (millions): 1.028
n_tokens_per_dead_feature_window (millions): 5.14
We will reset the sparsity calculation 861 times.
Number tokens in sparsity calculation window: 4.00e+03
Using Ghost Grads.
Run name: 32768-L1-8e-05-LR-0.0004-Tokens-3.447e+06
n_tokens_per_buffer (millions): 0.032896
Lower bound: n_contexts_per_buffer (millions): 0.000128
Total training steps: 861785
Total wandb updates: 8617
n_tokens_per_feature_sampling_window (millions): 1.028
n_tokens_per_dead_feature_window (millions): 5.14
We will reset the sparsity calculation 861 times.
Number tokens in sparsity calculation window: 4.00e+03
Using Ghost Grads.
{'n_layers': 24, 'd_model': 1024, 'd_head': 64, 'model_name': '', 'n_heads': 16, 'd_mlp': 4096, 'activation_name': 'gelu', 'eps': 1e-0

In [ ]:
AUTOENCODER_NAME = "laion_CLIP-ViT-L-14-DataComp.XL-s13B-b90K_blocks.23.hook_mlp_out_16384_layers_23"
sparse_autoencoder = sae_group.autoencoders[AUTOENCODER_NAME]
sparse_autoencoder = sparse_autoencoder.to(config.device)
layer_num = sparse_autoencoder.cfg.hook_point_layer
print(f"Chosen layer {layer_num} hook point {sparse_autoencoder.cfg.hook_point}")

NameError: name 'sae_group' is not defined

In [ ]:
import plotly.express as px

sparse_autoencoder.eval()  # prevents error if we're expecting a dead neuron mask for who grads
with torch.no_grad():
    batch_tokens, labels = activation_store.get_val_batch_tokens()
    _, cache = model.run_with_cache(batch_tokens)
    sae_out, feature_acts, loss, mse_loss, l1_loss, _ = sparse_autoencoder(
        cache[sparse_autoencoder.cfg.hook_point]
    )
    # del cache

    # ignore the bos token, get the number of features that activated in each token, averaged accross batch and position
    l0 = (feature_acts[:, 1:] > 0).float().sum(-1).detach()
    print("average l0", l0.mean().item())
    px.histogram(l0.flatten().cpu().numpy()).show()

NameError: name 'sparse_autoencoder' is not defined

In [ ]:
cache.keys()

In [7]:
del cache

In [6]:
import wandb
torch.set_grad_enabled(True)
from sae.train import train_sae_group_on_vision_model


if config.log_to_wandb:
    wandb.init(project=config.wandb_project, config=cast(Any, config), name=config.run_name)

train_sae_group_on_vision_model(
    model,
    sae_group,
    activation_store,
    train_contexts=None, #TODO load checkpoints correctly to match saelens v2.1.3 lm_runner!
    training_run_state=None,  #TODO load checkpoints correctly to match saelens v2.1.3 lm_runner!
    n_checkpoints=config.n_checkpoints,
    batch_size=config.train_batch_size_tokens,
    feature_sampling_window=config.feature_sampling_window,
    use_wandb=config.log_to_wandb,
    wandb_log_frequency=config.wandb_log_frequency,
    eval_every_n_wandb_logs=config.eval_every_n_wandb_logs,
    autocast=config.autocast,
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: willfulbytes. Use `wandb login --relogin` to force relogin








Objective value: 11732.4385:   4%|▍         | 4/100 [00:00<00:18,  5.14it/s]
/Users/aw/projects/vitsearch/.venv/lib/python3.11/site-packages/sae_lens/training/sparse_autoencoder.py:279: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  out = torch.tensor(origin, dtype=self.dtype, device=self.device)
/Users/aw/projects/vitsearch/.venv/lib/python3.11/site-packages/sae_lens/training/train_sae_on_language_model.py:611: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=autocast)


interrupted, saving progress
